# Tutorial 4 - A metrologically enabled pipeline.

In this tutorial we introduce the new metrologically enabled agents. We initialize an
agent, which generates an infinite sine signal. The signal is generated from an
external class `Signal` and delivers on each call one timestamp and one value each
with associated uncertainties.
 
The _MetrologicalSineGeneratorAgent_ is based on the new class
_agentMET4FOF.metrological_agents.MetrologicalAgent_. We only adapt the
methods `init_parameters()` and `agent_loop()`. This we need to hand over an instance
of the signal generating class and to generate the actual samples. The rest of the
buffering and plotting logic is encapsulated inside of the new base classes.

In [1]:
# %load tutorial_4_metrological_agents.py
from typing import Dict

from agentMET4FOF.agents import AgentNetwork
from agentMET4FOF.metrological_agents import MetrologicalAgent, MetrologicalMonitorAgent
from agentMET4FOF.metrological_streams import MetrologicalSineGenerator


class MetrologicalSineGeneratorAgent(MetrologicalAgent):
    """An agent streaming a sine signal

    Takes samples from the :py:mod:`SineGenerator` and pushes them sample by sample
    to connected agents via its output channel.
    """

    # The datatype of the stream will be SineGenerator.
    _sine_stream: MetrologicalSineGenerator

    def init_parameters(self, signal: MetrologicalSineGenerator = MetrologicalSineGenerator(), **kwargs):
        """Initialize the input data

        Initialize the input data stream as an instance of the
        :py:mod:`SineGenerator` class

        Parameters
        ----------
        signal : Signal
            the underlying signal for the generator
        """
        self._sine_stream = signal
        super().init_parameters()
        self.set_output_data(channel="default", metadata=self._sine_stream.metadata)

    def agent_loop(self):
        """Model the agent's behaviour

        On state *Running* the agent will extract sample by sample the input data
        streams content and push it via invoking
        :py:method:`AgentMET4FOF.send_output`.
        """
        if self.current_state == "Running":
            self.set_output_data(
                channel="default", data=[self._sine_stream._next_sample_generator()]
            )
            super().agent_loop()

    @property
    def metadata(self) -> Dict:
        return self._sine_stream.metadata.metadata


def main():

    # start agent network server
    agent_network = AgentNetwork(dashboard_modules=True)

    # Initialize signal generating class outside of agent framework.
    signal = MetrologicalSineGenerator()

    # Initialize metrologically enabled agent taking name from signal source metadata.
    source_name = signal.metadata.metadata["device_id"]
    source_agent = agent_network.add_agent(
        name=source_name, agentType=MetrologicalSineGeneratorAgent
    )
    source_agent.init_parameters(signal)

    # Initialize metrologically enabled plotting agent.
    monitor_agent = agent_network.add_agent(
        "MonitorAgent", agentType=MetrologicalMonitorAgent
    )

    # Bind agents.
    source_agent.bind_output(monitor_agent)

    # Set all agents states to "Running".
    agent_network.set_running_state()

    # Allow for shutting down the network after execution.
    return agent_network


if __name__ == "__main__":
    main()


Starting NameServer...
Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:3333 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:3333
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "agentMET4FOF.dashboard.Dashboard" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


INFO [2020-11-04 10:42:00.505031] (SineGenerator): INITIALIZED
INFO [2020-11-04 10:42:00.575146] (MonitorAgent): INITIALIZED
[2020-11-04 10:42:00.591467] (SineGenerator): Connected output module: MonitorAgent
SET STATE:   Running
[2020-11-04 10:42:01.511326] (SineGenerator): Pack time: 0.00013
[2020-11-04 10:42:01.511966] (SineGenerator): Sending: [array([[0.e+00, 1.e-09, 0.e+00, 5.e-01]]), <time_series_metadata.scheme.MetaData object at 0x7ffb103ad7c0>]
[2020-11-04 10:42:01.512496] (MonitorAgent): Received: {'from': 'SineGenerator', 'data': array([[0.e+00, 1.e-09, 0.e+00, 5.e-01]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7ffb102e5bb0>, 'senderType': 'MetrologicalSineGeneratorAgent', 'channel': 'default'}
[2020-11-04 10:42:01.512834] (MonitorAgent): Buffer: {'SineGenerator': {'data': array([[0.e+00, 1.e-09, 0.e+00, 5.e-01]]), 'metadata': [<time_series_metadata.scheme.MetaData object at 0x7ffb102e5bb0>]}}
[2020-11-04 10:42:01.512905] (MonitorAgent): Tproc: 0.00029

[2020-11-04 10:42:07.512192] (MonitorAgent): Tproc: 0.000501
[2020-11-04 10:42:08.511228] (SineGenerator): Pack time: 9.1e-05
[2020-11-04 10:42:08.511513] (SineGenerator): Sending: [array([[1.40000000e-02, 1.00000000e-09, 4.25779292e-01, 5.00000000e-01]]), <time_series_metadata.scheme.MetaData object at 0x7ffb103ad7c0>]
[2020-11-04 10:42:08.511654] (MonitorAgent): Received: {'from': 'SineGenerator', 'data': array([[1.40000000e-02, 1.00000000e-09, 4.25779292e-01, 5.00000000e-01]]), 'metadata': <time_series_metadata.scheme.MetaData object at 0x7ffb103aa160>, 'senderType': 'MetrologicalSineGeneratorAgent', 'channel': 'default'}
[2020-11-04 10:42:08.512084] (MonitorAgent): Buffer: {'SineGenerator': {'data': array([[0.00000000e+00, 1.00000000e-09, 0.00000000e+00, 5.00000000e-01],
       [2.00000000e-03, 1.00000000e-09, 6.27905195e-02, 5.00000000e-01],
       [4.00000000e-03, 1.00000000e-09, 1.25333234e-01, 5.00000000e-01],
       [6.00000000e-03, 1.00000000e-09, 1.87381315e-01, 5.00000000e-